### agent(generator), reflector, skill_manager(curator) 프롬프트

In [1]:
from ace.prompts_v2_1 import PromptManager
pm = PromptManager()
print("=== AGENT ===")
print(pm.get_agent_prompt())
print("\n=== REFLECTOR ===")
print(pm.get_reflector_prompt())
print("\n=== SKILL_MANAGER ===")
print(pm.get_skill_manager_prompt())

=== AGENT ===
# Identity and Metadata
You are ACE Agent v2.1, an expert problem-solving agent.
Prompt Version: 2.1.0
Current Date: 2026-01-05
Mode: Strategic Problem Solving with Skillbook Application

## Core Mission
You are an advanced problem-solving agent that applies accumulated strategic knowledge from the skillbook to solve problems and generate accurate, well-reasoned answers. Your success depends on methodical strategy application with transparent reasoning.

## Core Responsibilities
1. Apply accumulated skillbook strategies to solve problems
2. Show complete step-by-step reasoning with clear justification
3. Execute strategies to produce accurate, complete answers
4. Cite specific skills when applying strategic knowledge

## Skillbook Application Protocol

### Step 1: Analyze Available Strategies
Examine the skillbook and identify relevant skills:
{skillbook}

### Step 2: Consider Recent Reflection
Integrate learnings from recent analysis:
{reflection}

### Step 3: Process th

In [3]:
from ace.prompts_v2 import PromptManager
pm = PromptManager()
print("=== V2 AGENT ===")
print(pm.get_agent_prompt())
print("\n=== V2 REFLECTOR ===")
print(pm.get_reflector_prompt())
print("\n=== V2 SKILL_MANAGER ===")
print(pm.get_skill_manager_prompt())

=== V2 AGENT ===
# Identity and Metadata
You are ACE Agent v2.0, an expert problem-solving agent.
Prompt Version: 2.0.0
Current Date: 2026-01-01
Mode: Strategic Problem Solving
Confidence Threshold: 0.7

## Core Responsibilities
1. Analyze questions using accumulated skillbook strategies
2. Apply relevant skills with confidence scoring
3. Show step-by-step reasoning with clear justification
4. Produce accurate, complete answers

## Skillbook Application Protocol

### Step 1: Analyze Available Strategies
Examine the skillbook and identify relevant skills:
{skillbook}

### Step 2: Consider Recent Reflection
Integrate learnings from recent analysis:
{reflection}

### Step 3: Process the Question
Question: {question}
Additional Context: {context}

### Step 4: Generate Solution

Follow this EXACT procedure:

1. **Strategy Selection**
   - ONLY use skills with confidence > 0.7 relevance
   - NEVER apply conflicting strategies simultaneously
   - If no relevant skills exist, state "no_applica

/tmp/ipykernel_36600/1960639525.py:1: DeprecationWarning: prompts_v2 is deprecated and will be removed in a future version. Please use prompts_v2_1 instead for enhanced performance and features. See docs/PROMPTS.md for migration guide.
  from ace.prompts_v2 import PromptManager


In [5]:
from ace.prompts import AGENT_PROMPT, REFLECTOR_PROMPT, SKILL_MANAGER_PROMPT
print("=== V1 AGENT ===")
print(AGENT_PROMPT)
print("\n=== V1 REFLECTOR ===")
print(REFLECTOR_PROMPT)
print("\n=== V1 SKILL_MANAGER ===")
print(SKILL_MANAGER_PROMPT)

=== V1 AGENT ===
You are an expert assistant that must solve the task using the provided skillbook of strategies.
Apply relevant skills, avoid known mistakes, and show step-by-step reasoning.

Skillbook:
{skillbook}

Recent reflection:
{reflection}

Question:
{question}

Additional context:
{context}

Respond with a compact JSON object:
{{
  "reasoning": "<step-by-step chain of thought>",
  "skill_ids": ["<id1>", "<id2>", "..."],
  "final_answer": "<concise final answer>"
}}


=== V1 REFLECTOR ===
You are a senior reviewer diagnosing the agent's trajectory.
Use the skillbook, model reasoning, and feedback to identify mistakes and actionable insights.
Output must be a single valid JSON object. Do NOT include analysis text or explanations outside the JSON.
Begin the response with `{{` and end with `}}`.

Question:
{question}
Model reasoning:
{reasoning}
Model prediction: {prediction}
Ground truth (if available): {ground_truth}
Feedback: {feedback}
Skillbook excerpts consulted:
{skillbook

### Advanced Tutorial: Understanding ACE Internals (15 minutes)

In [7]:
from ace import OfflineACE, Agent, Reflector, SkillManager
from ace import LiteLLMClient, Sample, TaskEnvironment, EnvironmentResult


# Simple environment that checks if answer contains the ground truth
class SimpleEnvironment(TaskEnvironment):
    def evaluate(self, sample, agent_output):
        correct = str(sample.ground_truth).lower() in str(agent_output.final_answer).lower()
        return EnvironmentResult(
            feedback="Correct!" if correct else "Incorrect",
            ground_truth=sample.ground_truth
        )


# Initialize LLM client
client = LiteLLMClient(model="gpt-4o-mini")

# Create ACE components (three roles)
agent = Agent(client)              # Produces answers
reflector = Reflector(client)      # Analyzes performance
skill_manager = SkillManager(client)  # Updates skillbook

# Create adapter to orchestrate everything
adapter = OfflineACE(agent=agent, reflector=reflector, skill_manager=skill_manager)

# Create training samples
samples = [
    Sample(question="What is the capital of France?", context="", ground_truth="Paris"),
    Sample(question="What is 2 + 2?", context="", ground_truth="4"),
    Sample(question="Who wrote Romeo and Juliet?", context="", ground_truth="Shakespeare")
]

# Train the agent
print("Training agent...")
results = adapter.run(samples, SimpleEnvironment(), epochs=2)

# Save learned strategies
adapter.skillbook.save_to_file("my_agent.json")
print(f"✅ Agent trained! Learned {len(adapter.skillbook.skills())} strategies")

# Test with new question
test_output = agent.generate(
    question="What is 5 + 3?",
    context="",
    skillbook=adapter.skillbook
)
print(f"\nTest question: What is 5 + 3?")
print(f"Answer: {test_output.final_answer}")

INFO     [ace.llm_providers.instructor_client] Initialized InstructorClient with mode=Mode.MD_JSON, max_retries=3
INFO     [ace.llm_providers.instructor_client] Initialized InstructorClient with mode=Mode.MD_JSON, max_retries=3
INFO     [ace.llm_providers.instructor_client] Initialized InstructorClient with mode=Mode.MD_JSON, max_retries=3


12:39:38 - LiteLLM:INFO: utils.py:3476 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai


Training agent...
INFO     [LiteLLM] 
LiteLLM completion() model= gpt-4o-mini; provider = openai


12:39:42 - LiteLLM:INFO: utils.py:1331 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


...

INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler


12:41:45 - LiteLLM:INFO: utils.py:3476 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai


✅ Agent trained! Learned 4 strategies
INFO     [LiteLLM] 
LiteLLM completion() model= gpt-4o-mini; provider = openai


12:41:47 - LiteLLM:INFO: utils.py:1331 - Wrapper: Completed Call, calling success_handler


INFO     [LiteLLM] Wrapper: Completed Call, calling success_handler

Test question: What is 5 + 3?
Answer: 8


### Online Learning (Learn While Running)

In [8]:
from ace import OnlineACE

adapter = OnlineACE(
    skillbook=skillbook,
    agent=agent,
    reflector=reflector,
    skill_manager=skill_manager
)

# Process tasks one by one, learning from each
for task in tasks:
    result = adapter.process(task, environment)

NameError: name 'skillbook' is not defined

### roles.py Agent 클래스 테스트

In [2]:
# Cell 1: Dummy LLM + tests for Agent
from ace import Agent
from ace.skillbook import Skillbook

class DummyLLM:
    def __init__(self, responses):
        self.responses = list(responses)
        self.call_history = []

    def complete_structured(self, prompt, response_model, **kwargs):
        self.call_history.append(
            {"prompt": prompt, "kwargs": kwargs, "response_model": response_model}
        )
        if not self.responses:
            raise RuntimeError("No responses left in DummyLLM.")
        resp = self.responses.pop(0)
        if isinstance(resp, response_model):
            return resp
        if isinstance(resp, dict):
            return response_model(**resp)
        raise TypeError(f"Unsupported response type: {type(resp)}")

def run_agent_tests():
    # 1) Basic generation
    llm = DummyLLM([
        {"reasoning": "Step 1: add 2+2=4", "final_answer": "4", "skill_ids": []}
    ])
    agent = Agent(llm)
    skillbook = Skillbook()
    skillbook.as_prompt = lambda: "SKILLBOOK_TEST"
    out = agent.generate(
        question="What is 2+2?",
        context="Show your work",
        skillbook=skillbook,
    )
    assert out.final_answer == "4"
    assert "Step 1" in out.reasoning

    # 2) Skill ID extraction
    llm = DummyLLM([
        {"reasoning": "Using [math-00001], I computed 2+2=4", "final_answer": "4", "skill_ids": []}
    ])
    agent = Agent(llm)
    skillbook = Skillbook()
    skillbook.as_prompt = lambda: "SKILLBOOK_TEST"
    out = agent.generate(
        question="What is 2+2?",
        context="",
        skillbook=skillbook,
    )
    assert out.skill_ids == ["math-00001"]

    # 3) Prompt includes context/reflection/skillbook
    llm = DummyLLM([
        {"reasoning": "ok", "final_answer": "A", "skill_ids": []}
    ])
    agent = Agent(llm)
    skillbook = Skillbook()
    skillbook.as_prompt = lambda: "SKILLBOOK_TEST"
    _ = agent.generate(
        question="Q",
        context="Test context",
        reflection="Previous attempt was incorrect",
        skillbook=skillbook,
    )
    prompt = llm.call_history[0]["prompt"]
    assert "Additional Context: Test context" in prompt
    assert "Previous attempt was incorrect" in prompt
    assert "SKILLBOOK_TEST" in prompt

    print("Agent tests passed.")

run_agent_tests()


> /home/heodnjswns/agentic-context-engine/ace/roles.py(263)_generate_impl()
    261 
    262         import pdb; pdb.set_trace()
--> 263         return output
    264 
    265 

AgentOutput(reasoning='Step 1: add 2+2=4', final_answer='4', skill_ids=[], raw={})
*** AttributeError: 'AgentOutput' object has no attribute 'id'
*** AttributeError: 'AgentOutput' object has no attribute 'id'
[]


### roles.py Reflector 클래스 테스트

### roles.py SkillManager 클래스 테스트